# 네이버 뉴스 크롤링 - 구성 분석과 크롤링



## 네이버 뉴스 구성

sid1(대분류)와 sid2(중분류)를 사용하여 뉴스 분야를 지정합니다.

파라미터
* sid1(대분류): 100=정치, 101=경제, 102=사회, 103=생활문화, 104=세계, 105=IT/과학
* sid2(중분류)
  * 정치(sid1=100): 264=청와대, 265=국회/정당, 268=북한, 266=행정, 267=국방/외교, 269=정치일반
  * 경제(sid1=101): 258=증권, 261=산업/재계, 771=중기/벤처, 260=부동산, 262=글로벌 경제, 310=생활경제, 263=경제 일반
  * 사회(sid1=102): 249=사건사고, 250=교육, 251=노동, 254=언론, 252=환경, 59b=인권/복지, 255=식품/의료, 256=지역, 276=인물, 257=사회 일반
  * 생활문화(sid1=103):  241=건강정보, 239=자동차/시승기, 240=도로/교통, 237=여행/레저, 238=음식/맛집, 376=패션/뷰티, 242=공연/전시, 243=책, 244=종교, 248=날씨, 245=생활문화 일반
  * 세계(sid1=104): 231=아시아/호주, 232=미국/중남미, 233=유럽, 234=중동/아프리카, 322=세계 일반
  * IT/과학(sid1=105): 731=모바일, 226=인터넷/SNS, 227=통신/뉴미디어, 230=IT 일반, 732=보안/해킹, 283=컴퓨터, 229=게임/리뷰, 228=과학 일반
* date=20180110 날짜를 지정해서 특정 날짜의 목록



예를 들어, "경제 / 경제 일반" 영역의 경우 URL은 다음과 같습니다

https://news.naver.com/main/list.nhn?sid1=101&sid2=263&date=20190104&page=1


<img src="https://i.imgur.com/aKfkdhE.png">



# 필요 모듈 가져오기및 설정

In [ ]:
# !pip install -q newspaper3k

In [ ]:
import requests
from bs4 import BeautifulSoup

import dateutil
import io
import numpy as np
import pandas as pd

In [ ]:
# 구글 Colab 경우 (타임존 설정)
import os, time
from datetime import datetime
  
os.environ['TZ'] = 'Asia/Seoul'
time.tzset()

# 지정 영역 뉴스 목록 가져오기

"경제/경제 일반" (sid1=101, sid2=263) 영역의 뉴스 목록 가져오기

In [ ]:
r = requests.get('https://news.naver.com/')
r

In [ ]:
r = requests.get('https://news.naver.com/', headers={'user-agent':'chrome'})
r

뉴스 영역의 id

In [ ]:
# "경제/경제 일반" 페이지의 뉴스 목록 가져오기
import requests
from bs4 import BeautifulSoup

date, page = '20200101', '1'
url = f'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date={date}&page={page}'

r = requests.get(url, headers={'user-agent':'chrome'})
soup = BeautifulSoup(r.text)

<img src='https://i.imgur.com/FKzg6pz.png'>

In [ ]:
main_content = soup.find(id='main_content')
li_list = main_content.find_all('li')
len(li_list)

In [ ]:
li_list[0]

In [ ]:
li_list[1]

In [ ]:
# 날짜 파싱하기
import dateutil

date_str = '2019.01.04. 오후 11:17'
date_str = date_str.replace('오후', '') + ' PM' if '오후' in date_str else date_str
date_str = date_str.replace('오전', '') + ' AM' if '오전' in date_str else date_str

dateutil.parser.parse(date_str)

In [ ]:
li_list[2]

In [ ]:
li_list[3]

## 지정영역 목록

In [ ]:
# 전체 뉴스 목록 (동영상 뉴스 등을 제외)

for li in li_list:
    # 뉴스 개괄(제목, 링크, 서두, 날짜)
    try:
        dt = li.find('dt', attrs=None)
        link = dt.a['href']
        title = dt.text.strip()
        lede = li.find('span', attrs='lede').text
        writing = li.find('span', attrs='writing').text
        date_str = li.find('span', attrs='date').text
        date_str = date_str.replace('오후', '') + ' PM' if '오후' in date_str else date_str
        date_str = date_str.replace('오전', '') + ' AM' if '오전' in date_str else date_str
        date = dateutil.parser.parse(date_str)

        print(date, title)
        print(link)
    except Exception as e:
        pass

## 개별 뉴스 1건 내용 읽어오기

다음과 같은 뉴스 링크

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&sid2=263&oid=001&aid=0010563076



In [ ]:
url='https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&sid2=263&oid=001&aid=0010563076'

r = requests.get(url, headers={'user-agent':'chrome'})
soup = BeautifulSoup(r.text, 'html.parser')
soup.title.text

In [ ]:
r.status_code

<img src="https://i.imgur.com/r7tZIYD.png" >

### 실습 : 다음 문제를 수정해서 진행하세요.

브라우저 개발자 도구를 사용해서 뉴스 본문 페이지의 DIV 속성을 찾아 완성해 주세요.

In [ ]:
soup.find(id='articleBodyContents')

In [ ]:
article = soup.find(id='articleBodyContents')
article

#### 다음은 완성된 코드

In [ ]:
soup.find(id='                  ')#'articleBodyContents'

In [ ]:
article = soup.find(id='                        ') #'articleBodyContents')
article

### 텍스트만 추출

 - strip()

In [ ]:
article.text.strip()

## 함수로 정리 하기
개별 뉴스 1건의 제목과 본문 가져오기 

In [ ]:
## 함수로 정리

def get_naver_news_text(url):
    r = requests.get(url, headers={'user-agent':'chrome'})
    soup = BeautifulSoup(r.text, 'html.parser')
    # 제목 가져오기
    title = soup.title.text
    # 본문 가져오기
    article = soup.find(id='newsct_article')
    # article.script.extract()
    text = article.text.strip()
    return title, text

In [ ]:
## 함수 테스트

url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=366&aid=0000431718'
title, text = get_naver_news_text(url)

print(title)
print('-' * 100)
print(text)

## "실습" 함수를 날짜를 포함해 가져오도록 수정하세요.

## "실습" csv로 저장하세요.

In [ ]:
import csv

In [ ]:
with open('naver_news.csv', 'w') as f:
    csv.writer(f)
    csv.writerow(['제목', '내용'])
    csv.writerow([title, text])

# "실습" 위 지정역역 목록의 세션에서 뉴스 기사를 크롤링해서 CSV로 저장하세요

- 과제 폴더의 `11-22`일 폴더에 `이름.ipynb` 파일로 업로드 해주세요.

샘플:

```json
[
    { '2020-01-01 23:02:00',
     '아파트공급 ...유동자금',
     '.... 내용.... '
]
```


### 예) 아래는 예를 든 것임!!!

In [ ]:
news = []

for li in li_list:
    # 뉴스 개괄(제목, 링크, 서두, 날짜)
    try:
        dt = li.find('dt', attrs=None)
        link = dt.a['href']
        title = dt.text.strip()
        lede = li.find('span', attrs='lede').text
        writing = li.find('span', attrs='writing').text
        date_str = li.find('span', attrs='date').text
        date_str = date_str.replace('오후', '') + ' PM' if '오후' in date_str else date_str
        date_str = date_str.replace('오전', '') + ' AM' if '오전' in date_str else date_str
        date = dateutil.parser.parse(date_str)

        print(date, title)
        print(link)
        # ex)
        title, body = get_naver_news_text(link)
        
        news.append( (date, title, body)   )

    except Exception as e:
        pass

# "실습" 네이버 뉴스 `증권` 세션에서 10개 뉴스 기사를 크롤링해서 CSV로 저장하세요

- 과제 폴더의 `11-22`일 폴더에 `이름.ipynb` 파일로 업로드 해주세요.